In [1]:
import keras
from keras import layers
import numpy as np
import random
import io

2022-12-11 09:45:06.448540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-11 09:45:06.703955: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-11 09:45:06.703975: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-11 09:45:07.774870: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

600901/600901 [==============================] - 1s 2us/step
Corpus length: 600893
Total chars: 56
Number of sequences: 200285


/tmp/ipykernel_136402/1111328618.py:24: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
/tmp/ipykernel_136402/1111328618.py:25: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(64),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

2022-12-11 09:46:36.563551: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-11 09:46:36.563711: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-11 09:46:36.563732: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (yarburart-ThinkPad-E15): /proc/driver/nvidia/version does not exist
2022-12-11 09:46:36.564355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
epochs = 10
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

2022-12-11 09:55:31.110159: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 448638400 exceeds 10% of free system memory.


1565/1565 [==============================] - 101s 63ms/step - loss: 1.9013

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: " a school of distrust, still more of dis"
...Generated:  tine of the demore the in the something and the something and the himself and the so the something and the faction of the soul and the relaring and the soll of the soul and the something and the farst of the for the consiment of the saints and a soul and the solf--and the says and the soul and the for in the something and the faction of the into the enerary and the something and the something and 

...Diversity: 0.5
...Generating with seed: " a school of distrust, still more of dis"
...Generated:  trasice infleeration and the respiciamen for the sourther the for all the necessing and the himself a process of the so the happiness of the so this asmust and the intarken of soul religion and the liffedent as a soul and the ofting as the feeling for in every and and of the soul and it a

2022-12-11 09:58:34.081168: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 448638400 exceeds 10% of free system memory.


1565/1565 [==============================] - 96s 61ms/step - loss: 1.5614

Generating text after epoch: 1
...Diversity: 0.2
...Generating with seed: " the general destiny of the world and, n"
...Generated:  ot it is the stronger of the sense of the soul of the stronger of the strong of the stronger of the sense of the stronger of the subtless of the stronger of the strong and the strange of the stronger of the stronger of the moral extent of the stronger of the stronger of the stronger of the specte of the strong of the sense of the subtiment of the strong of the subtless of the strong of the sense o

...Diversity: 0.5
...Generating with seed: " the general destiny of the world and, n"
...Generated:  ot even in the terreed in a noulled as the same in the religious to the sense, which and become a passion of the suit, as manner of more that the soul in the more the sentiments of the self-good more in the strength of the standen subtle soul as to reason are power in this soul will is a s

2022-12-11 10:01:32.664703: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 448638400 exceeds 10% of free system memory.


1565/1565 [==============================] - 140s 89ms/step - loss: 1.4777

Generating text after epoch: 2
...Diversity: 0.2
...Generating with seed: "therto been treated by men like birds, w"
...Generated:  hich is an existence of the submic of the strength of the submic of the religion of the strength of the subjection of the strength of the strength of the strength of the strength of the sense of the strength of the existence of the strength of the submic of the strength of the most and the strength of the strength of the strength of the strength of the strength of the subjection of the most suffer

...Diversity: 0.5
...Generating with seed: "therto been treated by men like birds, w"
...Generated:  hich is sald interprets and antimalality in the higher out of the consciences on their conceple the cansure of its his strange, when the result of the former on former conception only that the open and which says that the existence of the suffer the really in his the belief and beacting t

2022-12-11 10:05:15.471006: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 448638400 exceeds 10% of free system memory.


  79/1565 [>.............................] - ETA: 21:26 - loss: 1.3904